## Cosine similarity

Redaction of PII in these steps:

- Select $n$ most probable subtitutions
- from these, extract $k$ most similar in terms of cosine similarity (hyper-sphere)
- select randomly

For multisubtoken words, nothing is said(?).
=> Trying by finding the most similar to the entire word. E.g.

- original = contex1 + "Amanda" + context2
- tokenized = "Aman" "da"
- redaction result = yes
- subtitution by token:
1. for context1 + [MASK] [MASK] + context2 the most probable predictions for the **first masked** are $S_1$
2. Find $S^r_1$ closest to "Amanda" where "Amanda" is max pooled form "Aman" and "da"
3. select $s_1$ in $S^r_1$ randomly
4. $S_2$ == most probable for context + s_1 + [MASK] + context2
5. Similarly find one closest to "Amanda" => s_2
6. Assume $s_1$+$s_2$ is a coherent word :)

Notes:

- if using the same model to redact and find substitutions: less computation
- if using different models, you may optimize both



In [1]:
from transformers import pipeline, pipelines, AutoTokenizer, AutoModelForPreTraining
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
from piifinder import PiiFinder

In [3]:
MODEL_NAME="TurkuNLP/bert-base-finnish-cased-v1"

In [4]:
# taking these from past course work:

def get_embed_for_full_dataset(dataset, lang, pipeline):
  model_name = MODEL_NAME
  #p=pipeline(task="feature-extraction",model=model_name,return_tensors=True,device=0)
  embedded=pipeline(pipelines.pt_utils.KeyDataset(dataset[lang], "text"), batch_size=64, truncation="only_first")

  # to cpu and take the mean over words
  embedded_pooled=[torch.mean(elem,axis=1).cpu() for elem in embedded]
  # to single matrix
  results=torch.vstack(embedded_pooled).numpy()
  return results

def get_embed_for_one_instance(x, pipeline):
    #p=pipeline(task="feature-extraction",model=MODEL_NAME,return_tensors=True,device=0)
    return pipeline(x)

def cosine_sim(x,y):
  M=cosine_similarity(x,y)
  aligned=np.argsort(-M,axis=-1)

  sims=[]
  for i in range(M.shape[0]): #M.shape[0] is the number of rows / input documents
    j=aligned[i,0] # index 1 for 2nd best match => first one if different languages
    score=M[i,j]
    sims.append((i,j,score))
  # sort in descending order  element -> score => sort by score
  sims.sort(key=lambda element:element[2],reverse=True)

  return sims

    

In [5]:
text = "My name is Leena and I like playing the piano"

model = AutoModelForPreTraining.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
pipe = pipeline(task="feature-extraction",model=MODEL_NAME,return_tensors=True,device=0)

In [6]:
pf = PiiFinder(model, tokenizer, 1e-4, "WordPiece")
output = pf.find_pii(text)

In [7]:
print(output)

{'decoded_text': '[CLS] My name is Leena and I like playing the piano [SEP]', 'tokenizer_output': {'input_ids': tensor([[  102,  9562, 37932, 50010,   645, 11266,  9624,   320, 21267, 50010,
         18463,  1365,  7217, 37837,   103]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, 'to_redact': [[5], [10, 11]]}


In [8]:
to_redact = output["to_redact"]
input = output["tokenizer_ouput"]

#to_redict = tok.input_ids[0][**redact_ids]
#to_substitute = tokenizer.decode(to_redact)
#good_predictions_1 = ["Mai", "An", "Emi", "Luci"]
#choose_index = 1 # An
#print(to_substitute)



SyntaxError: invalid syntax (861437718.py, line 3)

In [ ]:
# get embed
orig = get_embed_for_one_instance(to_substitute)
guesses = []

In [ ]:
print(orig)